In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
import bempp.api.shapes.fractals

In [2]:
N_wn = 20
lbound_wn = 0
ubound_wn = 10
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)

In [5]:
for dist in np.linspace(0.5, 3, 11):
    cube1 = bempp.api.shapes.cube(length=1, h = 0.05, origin=(-(dist*0.5 + 0.5)-0.5, -0.5, -0.5))
    cube2 = bempp.api.shapes.cube(length=1, h = 0.05, origin=((dist*0.5 + 0.5)-0.5, -0.5, -0.5))

    space_cube1 = bempp.api.function_space(cube1,'P',1)
    space_cube2 = bempp.api.function_space(cube2,'P',1)
    
    results = []
    for index2, point2 in enumerate(quad_points):

        wavenumber = -1j * np.log(point2)

        slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_cube1, space_cube1, space_cube1, wavenumber)
        slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_cube2, space_cube1, space_cube1, wavenumber)
        slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_cube1, space_cube2, space_cube2, wavenumber)
        slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_cube2, space_cube2, space_cube2, wavenumber)

        mat11 = slp11.weak_form().A
        mat12 = slp12.weak_form().A
        mat21 = slp21.weak_form().A
        mat22 = slp22.weak_form().A
        mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
        mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

        mat = np.block([[mat11,mat12],[mat21,mat22]])  
        mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
        mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

        inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                              [mat21_zero, mat22_inv]])  

        combined_inverse_free = inv_free_mat2.dot(mat)
        sign, logdet = slogdet(combined_inverse_free)

        results.append(logdet)
    print(dist, np.trapz(-np.array(results) / quad_points, quad_points) / (2 * np.pi))

0.5 0.08344738809037242
0.75 0.027339888950927094
1.0 0.013045483033998308
1.25 0.007355040966986678
1.5 0.004605357222520099
1.75 0.0030978353398840534
2.0 0.0021950349522245056
2.25 0.0016180700472230617
2.5 0.001230483572897415
2.75 0.0009596122035514893
3.0 0.0007641263465397668


In [ ]:
h = 0.1
0.5 0.08324333600429852
0.75 0.02729654839186554
1.0 0.013027739612061976
1.25 0.0073458725650954165
1.5 0.0046000294103323005
1.75 0.003094486968391024
2.0 0.0021927922826723706
2.25 0.0016164842887971896
2.5 0.001229316824014341
2.75 0.0009587286096210919
3.0 0.0007634396663346132

h = 0.05
0.5 0.08344738809037242
0.75 0.027339888950927094
1.0 0.013045483033998308
1.25 0.007355040966986678
1.5 0.004605357222520099
1.75 0.0030978353398840534
2.0 0.0021950349522245056
2.25 0.0016180700472230617
2.5 0.001230483572897415
2.75 0.0009596122035514893
3.0 0.0007641263465397668